# Homework 1

## 1.1
Using introspection alone to understand human cognitive processes would have limited usefulness as human cognitive 
processes are generally not conscious. For instance, trying to model the way a human throws a ball
purely through introspection would be almost impossible as so much of our physical action happens unconsciously.
Furthermore, It is common for people to not understand why they made a decision they made. Humans are often not
privy to their own cognitive processes making it difficult to use introspection to
gain much insight into our processes. When trying to make an AI that follows
the human thinking approach, introspection would have some usefulness but should probably be combined with other methods
of understanding the human mind.

## 1.2

Each state in my formulation is a list of cities, a path the salesman
would go along. Each path, even after actions done to it, should always
have the same starting and end point. The action the formulation can 
take is to swap two neighboring in the path cities in the order (as they are all interconnected).
In order to calculate the value, I am using the search.py graph class's get()
function which returns the distance between two nodes (which is also what I am using to place the cities randomly). 
I than multiply this by -1 to have the algorithms head towards the lowest value.



In [4]:
"""
This implements a local-search version of TSP for AIMA-Python.
@author: Noah Madrid
"""
from search import Problem, hill_climbing, simulated_annealing, exp_schedule, RandomGraph
import random

class TSP(Problem):
    """An implementation of TSP for local search.
        State representation:
            [city0, city1, city2]
        Move representation:
            [index0, index1]: Swap the city in the given path index with the city in the other index.
    """
    def __init__(self, initialPath, cityGraph):
        self.initial = initialPath
        self.cityGraph = cityGraph

    def actions(self, state):
        """Actions: swap two cities next to each other in the path (as long as the first and last city are not altered)
        """
        actions = []
        for i in range(1, len(state)-2):
            newPath = state[:]
            newPath[i], newPath[i+1] = newPath[i+1], newPath[i]
            actions.append(newPath)
        return actions

    def result(self, state, move):
        """Makes the given move."""
        return move

    def value(self, state):
        """This method computes a value of given state using distance traveled
        """
        value = 0;
        for i in range(len(state)-1):
            value = value + cityGraph.get(state[i],state[i+1])
        # multiply by one to pursue the lowest value
        return -1 * value

Describe your sample city-domain and how your code does on it.

In [5]:
if __name__ == '__main__':
    # Set the number of cities
    cityAmount = 20
    print('Cities:\t\t' + str(cityAmount))

    # Create unique city ids and add them to city list
    cityList = []
    for i in range(cityAmount):
        cityList.append(str(i))

    # Create a random graph using search RandomGraph()
    cityGraph = RandomGraph(cityList, cityAmount-1, 10000, 10000)

    # Initialize initial path randomly
    path = random.sample(cityGraph.nodes(), len(cityGraph.nodes()))

    # Add first city as the last city to make it the destination
    path.append(path[0])
    print('Starting path:\t' + str(path))

    p = TSP(path,cityGraph)

    # Solve the problem using hill climbing.
    hill_solution = hill_climbing(p)
    print('\nHill-climbing:')
    print('\tSolution:\t' + str(hill_solution))
    print('\tValue:\t\t' + str(p.value(hill_solution)))

    # Solve the problem using simulated annealing.
    annealing_solution = \
        simulated_annealing(p, exp_schedule(k=20, lam=0.005, limit=10000))
    print('\nSimulated annealing:')
    print('\tSolution:\t' + str(annealing_solution))
    print('\tValue:\t\t' + str(p.value(annealing_solution)))

Cities:		20
Starting path:	['10', '15', '1', '0', '12', '9', '7', '5', '19', '4', '14', '16', '8', '13', '6', '11', '2', '3', '18', '17', '10']

Hill-climbing:
	Solution:	['10', '1', '15', '0', '9', '7', '12', '5', '19', '4', '14', '8', '16', '13', '6', '11', '2', '17', '18', '3', '10']
	Value:		-113380

Simulated annealing:
	Solution:	['10', '1', '15', '12', '0', '9', '7', '5', '19', '4', '14', '8', '16', '13', '6', '11', '2', '17', '18', '3', '10']
	Value:		-120711


Both Problems result in an answer very quickly at small city numbers but generally hill climbing
results in a better answer (at least with a relatively small number of cities I have tested), 
possibly because this problem does not have very many local maximums.

## 1.3

Follow the same pattern for this problem.

In [6]:
print('Here\'s yet more code.')

Here's yet more code.
